# Generate truth resource files for CNVControlEventsQC

## Function definitions

In [ ]:
import pandas as pd
import numpy as np


def add_exon_idxs(df, exons):
        contigs = set(df.contig)
        for contig in contigs:
            df.loc[df.contig==contig,"start_exon_idx"]=np.searchsorted(exons.loc[exons.contig==contig].end,
                                                                       df.loc[df.contig==contig].start,"left")
            df.loc[df.contig==contig,"end_exon_idx"]=np.searchsorted(exons.loc[exons.contig==contig].start,
                                                                       df.loc[df.contig==contig].end,"right")
def get_exon_expanded_events(df, exons):
    add_exon_idxs(df, exons)
    df = df.loc[df.start_exon_idx != df.end_exon_idx].reset_index().astype({'start_exon_idx':int,'end_exon_idx':int})
    df_expanded = df.loc[df.index.repeat(df.end_exon_idx-df.start_exon_idx)]
    df_expanded['exon_idx'] = df_expanded.groupby(df_expanded.index).cumcount() + df_expanded.start_exon_idx
    df_expanded = df_expanded.set_index(df_expanded.contig + "_" + df_expanded.exon_idx.astype(str))
    df_expanded = df_expanded.join(exons[['start','end']], rsuffix='_exon')
    df_expanded['event_exon_start']=np.maximum(df_expanded.start, df_expanded.start_exon)
    df_expanded['event_exon_end']=np.minimum(df_expanded.end, df_expanded.end_exon)
    df_expanded = df_expanded.set_index(df_expanded.index + "_" + df_expanded.svtype)
    return df_expanded

def extract_end_from_info(info):
    infos = info.split(';')
    for i in infos:
        if i.startswith('END='):
            return i.replace('END=', '')    

def read_vcf_to_df(vcf_path):
    df = pd.read_csv(vcf_path, sep='\t',comment='#',
                         names=['contig','start','ID','REF','ALT','QUAL','FILTER','INFO','FORMAT','SAMPLE'], dtype={'start': int})
    
    df = df.query("ALT != '.' and FILTER=='PASS'").copy()
    df['end'] = df['INFO'].apply(extract_end_from_info).astype(int)
    df["svtype"] = df.ALT.str.replace("<","").str.replace(">","")
    return df[['contig', 'start', 'end', 'ALT', 'svtype', 'ID']]

def read_intervals_to_df(intervals_path):
    intervals = pd.read_csv(intervals_path, sep="\t", comment="@", names = ["contig","start","end","dummy1","dummy2"], dtype={'start': int, 'end': int})
    intervals['contig_idx'] = intervals.groupby('contig').cumcount()
    intervals = intervals.set_index(intervals.contig + "_" + intervals.contig_idx.astype(str))
    return intervals

def generate_truth(intervals_path, paths):
    intervals = read_intervals_to_df(intervals_path)

    control_events = pd.concat([read_vcf_to_df(path) for path in paths], ignore_index=True)
    control_events_expanded = get_exon_expanded_events(control_events, intervals)
    control_events_expanded = control_events_expanded.rename(columns={'index': 'event_id'})

    index_sample_counts = control_events_expanded.groupby(control_events_expanded.index).size()
    passing_indices = index_sample_counts[index_sample_counts >= 4].index
    filtered_control_events_expanded = control_events_expanded.loc[passing_indices]

    filtered_grouped_control_events = filtered_control_events_expanded.groupby(['contig', 'exon_idx', 'svtype']).agg({'event_exon_start': 'max', 'event_exon_end': 'min'}).reset_index()

    def group_and_aggregate_events(df):
        df = df.sort_values(by='exon_idx')
        df['group'] = (df['exon_idx'].diff().fillna(1) != 1).cumsum()

        aggregated = df.groupby('group').agg({
            'event_exon_start': 'min',
            'event_exon_end': 'max',
        }).reset_index(drop=True)

        return aggregated

    aggregated_events = filtered_grouped_control_events.groupby(['contig', 'svtype'], group_keys=True) \
        .apply(group_and_aggregate_events, include_groups=False).reset_index().drop(columns=['level_2']) \
        .rename(columns={'event_exon_start': 'start', 'event_exon_end': 'end'})
    
    aggregated_events = aggregated_events.sort_values(by=['contig', 'start', 'svtype'],
                                                      key=lambda x: x.astype(str).str.replace('chr', '').astype(int))

    return aggregated_events

## Execution

In [56]:
intervals_path = "gs://fc-9b3068ac-c9f3-48c2-b157-5d7e7286e32f/twist_hg38_targets.interval_list"

In [ ]:
paths_na12878 = ['gs://fc-863da545-cfc0-4c3f-b016-4ebc35f924b2/submissions/b90e5c81-7db4-4abe-8408-b1d078f4d738/CNVCallingAndMergeForFabric/c5a89822-b126-4f64-a769-794a2db2d4d5/call-SingleSampleGCNVAndFilterVCFs/SingleSampleGCNVAndFilterVCFs/d8f9fe2b-a079-4fae-88e9-fa6fe8d0e06b/call-FilterVCF/0437225738.filtered.genotyped-segments.vcf.gz',
 'gs://fc-863da545-cfc0-4c3f-b016-4ebc35f924b2/submissions/b90e5c81-7db4-4abe-8408-b1d078f4d738/CNVCallingAndMergeForFabric/19908fa6-c22b-40da-84c7-be6a6c151da5/call-SingleSampleGCNVAndFilterVCFs/SingleSampleGCNVAndFilterVCFs/b08b14a1-90f7-45ed-81dd-88e6f99c6e80/call-FilterVCF/0437225744.filtered.genotyped-segments.vcf.gz',
 'gs://fc-863da545-cfc0-4c3f-b016-4ebc35f924b2/submissions/b90e5c81-7db4-4abe-8408-b1d078f4d738/CNVCallingAndMergeForFabric/2610faa2-d735-4f13-a0ef-1ab481faa762/call-SingleSampleGCNVAndFilterVCFs/SingleSampleGCNVAndFilterVCFs/1168faac-ae1e-45b2-8c92-8d40f66e1c7f/call-FilterVCF/0437227355.filtered.genotyped-segments.vcf.gz',
 'gs://fc-863da545-cfc0-4c3f-b016-4ebc35f924b2/submissions/b90e5c81-7db4-4abe-8408-b1d078f4d738/CNVCallingAndMergeForFabric/504ce16a-0001-4f21-ba1c-effea2b92d7d/call-SingleSampleGCNVAndFilterVCFs/SingleSampleGCNVAndFilterVCFs/14c267a3-a4b8-4031-9513-14ecec6cbbc1/call-FilterVCF/0437227356.filtered.genotyped-segments.vcf.gz',
 'gs://fc-863da545-cfc0-4c3f-b016-4ebc35f924b2/submissions/b90e5c81-7db4-4abe-8408-b1d078f4d738/CNVCallingAndMergeForFabric/37a3ce6e-5d9c-4963-ae05-6f44331d8902/call-SingleSampleGCNVAndFilterVCFs/SingleSampleGCNVAndFilterVCFs/5a7c0cdf-d0da-4839-ac89-deaa02e2e288/call-FilterVCF/0437227363.filtered.genotyped-segments.vcf.gz']

truth_na12878 = generate_truth(intervals_path, paths_na12878)
truth_na12878.to_csv("truth_na12878.tsv", sep="\t", index=False)

In [ ]:
paths_na24385 = ['gs://fc-863da545-cfc0-4c3f-b016-4ebc35f924b2/submissions/491aa495-33c0-41a9-a7e5-bb564dede818/CNVCallingAndMergeForFabric/c6396060-71d2-465c-84b2-0c2c2b4809c3/call-SingleSampleGCNVAndFilterVCFs/SingleSampleGCNVAndFilterVCFs/9764b4cf-af73-4095-940e-3e482abe684b/call-FilterVCF/0437225751.filtered.genotyped-segments.vcf.gz',
 'gs://fc-863da545-cfc0-4c3f-b016-4ebc35f924b2/submissions/b90e5c81-7db4-4abe-8408-b1d078f4d738/CNVCallingAndMergeForFabric/a6305bbb-6e10-4439-89e3-7cabac97a4d9/call-SingleSampleGCNVAndFilterVCFs/SingleSampleGCNVAndFilterVCFs/68b32369-7fc9-4d09-9f84-ee0eb88f1780/call-FilterVCF/0437225752.filtered.genotyped-segments.vcf.gz',
 'gs://fc-863da545-cfc0-4c3f-b016-4ebc35f924b2/submissions/b90e5c81-7db4-4abe-8408-b1d078f4d738/CNVCallingAndMergeForFabric/a0121b59-1d7f-4622-be5a-0e34a927077a/call-SingleSampleGCNVAndFilterVCFs/SingleSampleGCNVAndFilterVCFs/244d1d8e-88d1-43f3-885f-76f4ba9fc1ab/call-FilterVCF/0437225753.filtered.genotyped-segments.vcf.gz',
 'gs://fc-863da545-cfc0-4c3f-b016-4ebc35f924b2/submissions/b90e5c81-7db4-4abe-8408-b1d078f4d738/CNVCallingAndMergeForFabric/abb9ed4e-8fcd-44cb-b196-e3ae16bc166a/call-SingleSampleGCNVAndFilterVCFs/SingleSampleGCNVAndFilterVCFs/6f8e7059-cfc3-4684-b88f-27c4c864d29b/call-FilterVCF/0437225754.filtered.genotyped-segments.vcf.gz',
 'gs://fc-863da545-cfc0-4c3f-b016-4ebc35f924b2/submissions/491aa495-33c0-41a9-a7e5-bb564dede818/CNVCallingAndMergeForFabric/82f0ad40-3db5-4bb3-8f28-468655d8231b/call-SingleSampleGCNVAndFilterVCFs/SingleSampleGCNVAndFilterVCFs/8289dd66-d420-4559-8161-5756121c8776/call-FilterVCF/0437225755.filtered.genotyped-segments.vcf.gz']

truth_na24385 = generate_truth(intervals_path, paths_na24385)
truth_na24385.to_csv("truth_na24385.tsv", sep="\t", index=False)